# Objective

In this notebook, we shall study how we can examine the vegetation cover of a region with the help of satellite data. This notebook aims to familiarise with the concept of satellite imagery data and how it can be analyzed to investigate real-world environmental and humanitarian challenges.

# Region of Interest

I was particularly interested in knowing about the vegetation density in Southern Pakistan . Therefore, the dataset in this article pertains to that area. However, the analysis would remain the same for any area in the world.

![](area_of_interest.png)


# Satellite Imagery: An Overview

Satellite Imagery is the image of Earth(or other planets) which are collected by imaging satellites. Governments or private firms may own these Satellites. Satellite imaging companies sell images by licensing them to governments and businesses such as Apple Maps and Google Maps.

# Getting the Data

For this particular case study, we will be working with the Surface Reflectance (SR) Data. Simply put, the SR data is that satellite data which has been algorithmically corrected to remove any interference from the atmosphere. Let’s search & download some imagery of area around southern Pakistan.

The data used in this exercise has been downloaded from Planet Explorer. Planet Explorer is a product of Product labs and is used to explore daily imagery right in our browser. Planet labs operate the largest fleet of Earth-imaging satellites, and the data provided by them is used for monitoring vegetation to measuring agriculture outputs.

# Imports

In [ ]:
import os
import math
import json
import numpy
import requests
import rasterio
from matplotlib import colors
import matplotlib.pyplot as plt
from clint.textui import progress
from requests.auth import HTTPBasicAuth

# Downloading satellite images from Planet

## Our coordinates data of our area of interest

This can be obtained from http://geojson.io

In [ ]:
geojson_geometry = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          72.47406005859375,
          31.276203454458123
        ],
        [
          74.46807861328125,
          31.276203454458123
        ],
        [
          74.46807861328125,
          32.27087780256757
        ],
        [
          72.47406005859375,
          32.27087780256757
        ],
        [
          72.47406005859375,
          31.276203454458123
        ]
      ]
    ]
}

## Filters for downloading satellite images

In [ ]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2016-08-31T00:00:00.000Z",
    "lte": "2016-09-01T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [ ]:
PLANET_API_KEY = "PL_API_KEY" # replace PL_API_KEY with Planet API key in quotes

item_type = "PSScene4Band"

# API request object
search_request = {
  "interval": "day",
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

print(json.dumps(search_result.json(), indent=1))

## Downloading the Image

In [ ]:
item_id = search_result.json()["features"][0]["id"]

session = requests.Session()
session.auth = (PLANET_API_KEY, '')

item = \
  session.get(
    ("https://api.planet.com/data/v1/item-types/" +
    "{}/items/{}/assets/").format(item_type, item_id))

links = item.json()[u"analytic"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

In [ ]:
# Request activation of the 'analytic' asset:
activate_result = session.get(activation_link)
  
activation_status_result = session.get(self_link)
    
print(activation_status_result.json()["status"])

When the activation status changes to “active” from “inactive”,”we can download the image in .tiff format.

In [ ]:
download_link = activation_status_result.json()["location"]
print(download_link)

# Exploring the Satellite Imagery

The python’s Rasterio library makes it very easy to explore satellite images. Satellite Images are nothing but grids of pixel-values and hence can be interpreted as multidimensional arrays.

## Importing Image

In [ ]:
image_file = "image.tif"
sat_data = rasterio.open(image_file)

## Calculating the dimensions of the image on earth in metres

In [ ]:
width_in_projected_units = sat_data.bounds.right - sat_data.bounds.left
height_in_projected_units = sat_data.bounds.top - sat_data.bounds.bottom

print("Width: {}, Height: {}".format(width_in_projected_units, height_in_projected_units))

## Rows and Columns

In [ ]:
print("Rows: {}, Columns: {}".format(sat_data.height, sat_data.width))

## Converting the pixel co-ordinates to longitudes and latitudes

In [ ]:
# Upper left pixel
row_min = 0
col_min = 0

# Lower right pixel.  Rows and columns are zero indexing.
row_max = sat_data.height - 1
col_max = sat_data.width - 1

# Transform coordinates with the dataset's affine transformation.
topleft = sat_data.transform * (row_min, col_min)
botright = sat_data.transform * (row_max, col_max)

print("Top left corner coordinates: {}".format(topleft))
print("Bottom right corner coordinates: {}".format(botright))

# Bands

The image that we are inspecting is a multispectral image consisting of 4 bands int he order B,G,R,N where N stands for near infrared.each band is stored as a numpy array.

In [ ]:
print(sat_data.count)

# sequence of band indexes
print(sat_data.indexes)

# Visualising the Satellite Imagery

In [ ]:
# Load the 4 bands into 2d arrays - recall that we previously learned PlanetScope band order is BGRN.
b, g, r, n = sat_data.read()

In [ ]:
# Displaying the blue band.

fig = plt.imshow(b)
plt.show()

In [ ]:
# Displaying the green band.

fig = plt.imshow(g)
fig.set_cmap('gist_earth')
plt.show()

In [ ]:
# Displaying the red band.

fig = plt.imshow(r)
fig.set_cmap('inferno')
plt.colorbar()
plt.show()

In [ ]:
# Displaying the infrared band.

fig = plt.imshow(n)
fig.set_cmap('winter')
plt.colorbar()
plt.show()

# Vegetation Index calculation from Satellite Imagery


Figure showing the changes in NDVI with the changing seasons.
![Figure showing the changes in NDVI with the changing seasons.](1_lnpMpXnN2R_nBbnVYedrdQ.gif)


## Vegetation Index

A vegetation index is an indicator of the greenness of any area. It is a measure to monitor the health of a vegetation. A variety of data is captured by satellite sensors and one such type of data specifically measures wavelengths of light absorbed and reflected by green plants.

Dense vegetation reflects a lot of near-infrared light(not visible to the human eye) as compared to the visible red light. The reverse happens in case of sparse vegetation. Thus, as a plant canopy changes from early spring growth to late-season maturity and senescence, these reflectance properties also change.

## NDVI

One of the most widely used index to measure vegetation is the Normalized Difference Vegetation Index (NDVI). the NDVI values range from +1.0 to -1.0. It was developed by NASA scientist Compton Tucker in 1977 and is derived from satellite imagery. It can be expressed as follows.

![NDVI](ndvi.png)

NDVI compares the reflected near-infrared light to reflected visible red light, by the plants.

![NDVI](comparison.png)

## Benefits of NDVI 

1. The NDVI values give a rough estimation of the type, amount and condition of a vegetation at a place which is very useful fo researchers.
2. NDVI values can also be averaged over time to establish “normal” growing conditions in a region for a given time of year. This primarily helps in identifying areas where there are changes in vegetation due to human activities such as deforestation, natural disturbances such as wildfires, or changes in plants’ phenological stage.

## Calculating NDVI for our Area of Interest

We already have our downloaded data in the form of a .tiff image. In this section, we shall calculate and NDVI index and analyse it.

### Extracting the data from the red and near-infrared bands

In [ ]:
filename = 'image.tif'
with rasterio.open(filename) as src:
    band_red = src.read(3)
with rasterio.open(filename) as src:
    band_nir = src.read(4)

### Calculating NDVI

In [ ]:
# Do not display error when divided by zero 

numpy.seterr(divide='ignore', invalid='ignore')

# NDVI 
ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir + band_red)

### Checking the range of NDVI values(excluding NaN)

Let's print the minimum and maximum values in our calculated ndvi. Because we're using the NDVI formula to normalize the input bands, we know that our expected values should fall within -1.0 to +1.0.

In [ ]:
print(numpy.nanmin(ndvi)) 
print(numpy.nanmax(ndvi))

### Saving the NDVI image

We will save the results to a new single band image. This new image file will use the geospatial data rom the original geotiff image.

In [ ]:
# get the metadata of original GeoTIFF:
meta = src.meta
print(meta)

# get the dtype of our NDVI array:
ndvi_dtype = ndvi.dtype
print(ndvi_dtype)

# set the source metadata as kwargs we'll use to write the new data:
kwargs = meta

# update the 'dtype' value to match our NDVI array's dtype:
kwargs.update(dtype=ndvi_dtype)

# update the 'count' value since our output will no longer be a 4-band image:
kwargs.update(count=1)

# Finally, use rasterio to write new raster file 'data/ndvi.tif':
with rasterio.open('ndvi.tif', 'w', **kwargs) as dst:
        dst.write(ndvi, 1)

### Applying a color scheme to visualize the NDVI values on the new image

The values in our NDVI will range from -1 to 1. To best visualize this, we want to use a diverging color scheme, and we want to center the colorbar at a defined midpoint.

In [ ]:
class MidpointNormalize(colors.Normalize):
   
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
       
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return numpy.ma.masked_array(numpy.interp(value, x, y), numpy.isnan(value))

### Interpretation of NDVI

In [ ]:
# Set min/max values from NDVI range for image

min=numpy.nanmin(ndvi)
max=numpy.nanmax(ndvi)

# Set our custom midpoint for most effective NDVI analysis
mid=0.1

# Setting color scheme ref:https://matplotlib.org/users/colormaps.html as a reference
colormap = plt.cm.RdYlGn 
norm = MidpointNormalize(vmin=min, vmax=max, midpoint=mid)
fig = plt.figure(figsize=(20,10))


ax = fig.add_subplot(111)

# Use 'imshow' to specify the input data, colormap, min, max, and norm for the colorbar
cbar_plot = ax.imshow(ndvi, cmap=colormap, vmin=min, vmax=max, norm=norm)


# Turn off the display of axis labels 
ax.axis('off')

# Set a title 
ax.set_title('Normalized Difference Vegetation Index', fontsize=17, fontweight='bold')

# Configure the colorbar
cbar = fig.colorbar(cbar_plot, orientation='horizontal', shrink=0.65)

# Call 'savefig' to save this plot to an image file
fig.savefig("ndvi-image.png", dpi=200, bbox_inches='tight', pad_inches=0.7)

# let's visualize
plt.show()

### Generating a histogram of NDVI values

A Histogram or any other chart can be useful for quick analysis by giving a visual insight into the distribution of "healthy" vs "unhealthy" vegetation values in your study area.

In [ ]:
# Define a new figure
fig2 = plt.figure(figsize=(20,10))

# Give this new figure a subplot, which will contain the histogram itself
ax = fig2.add_subplot(111)

# Add a title & (x,y) labels to the plot
plt.title("NDVI Histogram", fontsize=18, fontweight='bold')
plt.xlabel("NDVI values", fontsize=14)
plt.ylabel("Number of pixels", fontsize=14)


# For the x-axis, we want to count every pixel that is not an empty value
x = ndvi[~numpy.isnan(ndvi)]
color = 'g'
# call 'hist` with our x-axis, bins, and color details
ax.hist(x,bins=30,color=color,histtype='bar', ec='black')

# Save the generated figure to an external image file
#fig2.savefig("ndvi-histogram.png", dpi=200, bbox_inches='tight', pad_inches=0.5)


plt.show()

# Conclusion

The satellite imagery data can be analysed over a period of time to understand the causes of the decline in vegetation for a region. Similarly, the analysis can also enable us to point out if there has been severe deforestation in any area which might be leading to effects of global warming. Prediction of hurricanes, droughts and floods are other areas where analysis of satellite imagery is being extensively applied. There is no better way to use technology than to work on some real problems threatening the planet and being able to utilise data from the satellites is a step in that direction.